In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import scipy.constants as sc

In [50]:
#Constats
#Material
decanol1 = {'m':158.285,
            'n_atom':33,
            'DOF':99,
            'rho_l':830,
            'c_l': 1600,
            'p*':1.3,
            'T*':298.15,
            'deltaH_vap':8.1*10**4}

#Chamber
chamber = {'Volume V_ch':10**(-2),
           'Surface area A_ch':0.7,
           'Pumping speed S_p':50*10**(-3),
           'wall leak flux j_w':3*10**12,
           'liquid vol V_l':1*10**(-6),
           'surface area A_l':4.66*10**(-4)}

#Set parameters in use
m, n_atom, D, rho_l, c_l, p_star, T_star, deltaH_vap = decanol1.values()
V_ch, A_ch, S_p, j_w, V_l, A_l = chamber.values()
t_s = 10**3 #tau_s


#Start values
T_l = 305
P_v = 0.7
T_v = 298

In [51]:
def P_sat(T):
    if T <= T_star:
        raise ValueError('T must be bigger than T*')
    else:
        return p_star * np.log(- deltaH_vap / (sc.k * sc.N_A) * (1/T - 1/T_star))

def j_ev():
    sigma_e = P_sat(T_l) / P_v * np.exp( (D+4) * (1 - T_v/T_l) ) * (T_v/T_l)**(D+4)
    sigma_c = np.sqrt(T_v/T_l) * np.exp( -(D+4) * (1 - T_v/T_l) ) * (T_v/T_l)**(-(D+4))
    
    return np.sqrt(1/(2*np.pi*m*sc.k)) * ( sigma_e * P_sat(T_l)/np.sqrt(T_l) - sigma_c * P_v/np.sqrt(T_l) )

def delta_valve(t):
    if t > 10:
        return 0
    else:
        return 1

def dpdt(j_ev, delta_valve):
    print(A_l/V_ch * sc.k * T_v * j_ev)
    print(A_ch/V_ch * sc.k * T_v * j_w)
    print(- S_p/V_ch * P_v * delta_valve)
    return A_l/V_ch * sc.k * T_v * j_ev + A_ch/V_ch * sc.k * T_v * j_w - S_p/V_ch * P_v * delta_valve

def dTdt(j_ev):
    return - ( A_l * deltaH_vap) / ( sc.N_A * rho_l * V_l * c_l) * j_ev + ( T_v - T_l) / t_s

def dVdt(j_ev):
    return - m/(rho_l * A_l) * j_ev

In [52]:
pressure = [P_v]
liquid_temparature = [T_l]
liquid_volume = [V_l]
time = np.linspace(10**(-1), 10**4, 1000000)

for t in time:
    #Calc j_ev
    j = j_ev()

    valve = delta_valve(t)

    #Calculate change
    dp = dpdt(j, valve)
    dT = dTdt(j)
    dV = dVdt(j)

    print(P_v,T_l,V_l)
    #Add change to value
    # P_v += dp
    # T_l += dT
    # V_l += dV

    #Add to list
    pressure.append(P_v)
    liquid_temparature.append(T_l)
    liquid_volume.append(V_l)

    
    print(dp,dT,dV, valve,j)
    break



-4.5563934300828864e-14
8.640101442000001e-07
-3.5
0.7 305 1e-06
-3.4999991359899014 -0.006999999999988783 97254902998.02599 1 -237648869.95973402


In [41]:
pressure


[0.005, 0.005]

In [25]:
D

99